This NB goes through the reflection algorithm in the reflection layer.

In [ ]:
import numpy as np
import tensorflow as tf

# Check tensor shapes

Go through the algorithm with some dummy data to check broadcasting, shapes, etc.. Compare to <code>src/reflection_layer.py</code>.

## Prepare dummy data

In [ ]:
# set up array (tensor) shapes for experiments
data_shape = (50, 8, 8, 3)  # shape of the full data set; e.g.: 50 8x8 RGB images 
sample_shape = data_shape[1:]  # shape of a single sample
print('Sample shape:', sample_shape)

In [ ]:
# define reflector (ie kernel)
reflector = np.random.rand(*sample_shape)
print('Reflector shape:', reflector.shape)

In [ ]:
# define data
x = np.random.rand(*data_shape)
print('Data shape:', x.shape)

## Steps of the algorithm

In [ ]:
# elementwise product of reflector and data (careful: broadcasting!)
ewp = tf.multiply(x,reflector) 
ewp.shape

In [ ]:
# reduce sum of ewp gives vector of scalar products: samples dot reflector
scalars = tf.reduce_sum(ewp, axis=tuple(range(1, len(x.shape))), keepdims=True)
scalars.shape

In [ ]:
# expand dimensions of reflector kernel so that it can be multiplied with scalars
expanded_reflector = tf.expand_dims(reflector, 0)
expanded_reflector.shape

In [ ]:
# the outpusts of the last two cells can be multiplied (broadcasting!)
prod = tf.multiply(expanded_reflector, scalars)
prod.shape

In [ ]:
# final output: reflection of data x across the orthogonal complement of reflector
output = (2/ tf.reduce_sum(tf.multiply(reflector, reflector))) * prod - x
output.shape

# Explicit example

Check for an explicit example that the algorithm does the right thing.

## Function

In [ ]:
def reflect(x, reflector):
    # elementwise product of reflector and data (careful: broadcasting!)
    ewp = tf.multiply(x,reflector)
    # reduce sum of ewp gives vector of scalar products: samples dot reflector
    scalars = tf.reduce_sum(ewp, axis=tuple(range(1, len(x.shape))), keepdims=True)
    # expand dimensions of reflector kernel so that it can be multiplied with scalars
    expanded_reflector = tf.expand_dims(reflector, 0)
    # the outpusts of the last two cells can be multiplied (broadcasting!)
    prod = np.multiply(expanded_reflector, scalars)
    # final output: reflection of data x across the orthogonal complement of reflector
    return (2/ tf.reduce_sum(tf.multiply(reflector, reflector))) * prod - x

## Define reflector and data

For an explicit example, let's look at the simple case where each sample has shape <code>(3,)</code> i.e. is an vector in R3. As the kernel/weights/reflector, let's use <code>[1,0,0]</code> -- the unit vector along the x-axis. Its orthogonal complement (the set of vectors that are orthogonal to it) is the yz-plane. Reflecting a vector <code>(a,b,c)</code> across that plane gives <code>(-a,b,c)</code>. In the implementation of the reflection layer, this is followed by an overall multiplication by -1. We end up with <code>(a,-b,-c)</code>. Check in the output below that this is indeep what the code produces.

In [ ]:
reflector_tf = tf.convert_to_tensor(np.array([1,0,0]), dtype=tf.float32)
x_tf = tf.convert_to_tensor(np.array([[1,0,0],[0,1,0],[0,0,1],[1,1,1]]), dtype=tf.float32)

## Result

In [ ]:
print('\nAs the reflection of\n')
print(x_tf)
print('\nacross the orthogonal complement of\n')
print(reflector_tf)
print('\n(followed by an overall multiplication by -1),')
print('the algorithm obtains\n')
print(reflect(x_tf, reflector_tf))